# Approximation Methods

> “An approximate answer to the right problem is worth a good deal more than
an exact answer to an approximate problem.” -- John Tukey

Markov chain Monte Carlo (MCMC) is the *de facto* standard for the estimation of Bayesian models. It is an important and useful approach because it is asymptotically exact and can be implemented readily in software and applied to a wide range of probabilistic models. The main drawback of MCMC, however, is its **computational expense**, as it requires repeated calculation of likelihoods and other quantities at every iteration of the algorithm. These calculations typically involve all of the data specified in the model, and hence do not scale well with the size of the dataset being used to fit the model.

An alternative to this employs one of several *approximation* methods. By an approximation, we are here referring to methods that do not exactly calculate or sample from the full posterior distribution specified by the model, but rather, either returns one or more moments of the posterior or use an alternative functional form in place of the true posterior distribution. 

We will outline two of these methods that are available in PyMC3.

## Maximum a posteriori (MAP) estimation

The most straightforward way for obtaining estimates from a Bayesian model is to find the maximum *a posteriori* estimate of the model parameters. This simply involves applying a numerical optimization algorithm to the model, several of which are available in the SciPy package for Python. Since the marginal likelihood is a constant with respect to the parameters, the estimates of the parameters derived from a non-normalized model will be the same as those from a normalized model. 

$$\hat{\theta}_{MAP}(y) = \text{argmax}_{\theta} \frac{Pr(y|\theta)Pr(\theta)}{\int Pr(y|\theta)Pr(\theta) d\theta} = \text{argmax}_{\theta} Pr(y|\theta)Pr(\theta) $$

Let's use MAP to obtain estimates for the survival model that we introduced previously.

In [ ]:
%load ../data/melanoma_data.py

In [ ]:
from pymc3 import Normal, Model, DensityDist, sample
from pymc3.math import log, exp

with Model() as melanoma_survival:

    # Convert censoring indicators to indicators for failure event
    failure = (censored==0).astype(int)

    # Parameters (intercept and treatment effect) for survival rate
    beta = Normal('beta', mu=0.0, sd=1e5, shape=2)

    # Survival rates, as a function of treatment
    lam = exp(beta[0] + beta[1]*treat)
    
    # Survival likelihood, accounting for censoring
    def logp(failure, value):
        return (failure * log(lam) - lam * value).sum()

    x = DensityDist('x', logp, observed={'failure':failure, 'value':t})



The MAP estimate can be obtained in PyMC3 via the `find_MAP` function. As with `sample`, we run `find_MAP` inside a model context, or pass the model explicitly to the function as the `model` parameter.

Starting values can be optionally passed as a `dict` to the `start` parameter. By default, `fmin_MAP` uses SciPy's `fmin_bfgs` function to find the maximum, which is an implementation of the [Broyden–Fletcher–Goldfarb–Shanno algorithm](https://en.wikipedia.org/wiki/Broyden–Fletcher–Goldfarb–Shanno_algorithm). If there are discrete variables in the model, then `fmin_powell` is used, which is SciPy's implementation of [Powell's method](https://en.wikipedia.org/wiki/Powell%27s_method), a more general algorithm.

In [ ]:
from pymc3 import find_MAP

with melanoma_survival:
    estimates = find_MAP()

In [ ]:
estimates

For this model, the MAP estimates are comparable to those we would have obtained using MCMC sampling:

In [ ]:
from pymc3 import sample

with melanoma_survival:
    trace = sample(1000, init=None)

In [ ]:
from pymc3 import summary

summary(trace)

`find_MAP` only returns estimates unobserved random variables from the model, and does not include deterministic values. If we wish to evaluate a determinsitic quantity, we can construct a Theano function and pass in the relevant parameter values as arguments.

The major limitation to using MAP for inference is that there is no associated measure of uncertainty. Hence, `find_MAP` cannot be used for inference. It is useful, however, for getting a sense of typical values the model may take for a particular dataset, and for PyMC3 it is intended to be used to get reasonable starting values for use in MCMC algorithms.

## Variational Inference

An alternative approach to approximating the posterior disstribution that is difficult to calculate analytically is to perform inference on an **appoximation to the true posterior distribution**. 

The idea is to choose a convenient approximating density $q(\theta, \phi)$, with vector of corresponding parameters $\phi$. The goal is to select $\phi$ such that $q(\theta, \phi)$ is as similar as possible to the true posterior. We therefore require a loss function that measures the similarity of $q(\theta, \phi)$ to $p(\theta| y)$.

The loss function employed by variational inference is the **Kullback-Leibler distance**:

$$\text{KL}[q(\theta, \phi) || p(\theta| y)] = \int q(\theta, \phi) \frac{q(\theta, \phi)}{p(\theta| y)} d\theta$$

However, this integral is difficult to work with, so instead a proxy to KL, called the evidence lower bound, is minimized instead:

$$ELBO = \mathbb{E}_{q(\theta)} [\log p(y, \theta)] − \mathbb{E}_{q(θ)} [\log q(\theta, \phi)]$$

The first term of the ELBO expression $\mathbb{E}_{q(\theta)} [\log p(y, \theta)]$ is the expectation of the log joint density under the approximation, while the second term $\mathbb{E}_{q(θ)} [\log q(\theta, \phi)]$ is called the *entropy* of the variational approximation.

Algorithms for performing variational inference are difficult to construct, and this has limited its adoption for applications.

### Automatic Differentiation Variational Inference

Kucukelbir *et al.* (2015) devised a method for automating the variational inference approach, by making a **flexible choice** for the approximating distribution, and transforming the latent variables to an **unconstrained coordinate space** before fitting the model.

ADVI proceeds in three steps:

1. Transform the model's latent variables to the real coordinate space
2. Specify a normal variational distribution.
3. Maximize the variational objective via automatic differentiation and stochastic optimization

The ADVI procedure works for differentiable probability models (*i.e.* those comprised of continuous latent variables) only. This is because it requires the calculation of the gradient of the log-joint with respect to the stochastic variables:

$$\nabla_{\theta} \log p(y, \theta)$$

The key to making ADVI work is the transformation of constrained parameters to the unconstrained, real coordinate space. This allows us to use a Gaussian distribution as the variational density. As with the classical variational inference algorithm, we impose the **mean field assumption**, whereby the Gaussian distributions over all the parameters can be fully factorized:

$$q(\zeta, \phi) = \prod_{j=1}^J N(\zeta | \mu_j, \sigma_j^2)$$

where $\zeta$ are the parameters after tranformation by $T: \theta \rightarrow \zeta$.

The inverse of the transform used (*e.g.* log for positive variables, logit for probabilities) and the associated Jacobian allows for a **non-normal variational approximation** on the support of the original variable.

In [ ]:
import numpy as np

np.random.seed(20090425)
data = np.random.randn(100)

In [ ]:
import pymc3 as pm

with pm.Model() as model: 
    μ = pm.Normal('μ', mu=0, sd=1, testval=0)
    σ = pm.HalfNormal('σ', sd=1)
    n = pm.Normal('n', mu=μ, sd=σ, observed=data)

In [ ]:
with model:
    
    means, sds, elbos = pm.advi(n=10000, accurate_elbo=True)

In [ ]:
means

In [ ]:
with model:
    trace = pm.sample(1000, init=None)

In [ ]:
%matplotlib inline
import seaborn as sns
from scipy import stats

ax = sns.distplot(trace['μ'], label='NUTS')
xlim = ax.get_xlim()
x = np.linspace(xlim[0], xlim[1], 100)
y = stats.norm(means['μ'], sds['μ']).pdf(x)
ax.plot(x, y, label='ADVI')
ax.set_title('μ')
ax.legend(loc=0);

## Example: Inference of Gaussian mixture model with mini-batch ADVI

Here is a more realistic application of ADVI, to the estimation of a **Gaussian mixture model**. 

We can generate some artificial data from a mixuture of two Gaussian components.  

In [ ]:
from pymc3 import Normal, Metropolis, sample, MvNormal, Dirichlet, Model, DensityDist, find_MAP, NUTS, Slice
import theano.tensor as tt
from theano.tensor.nlinalg import det
import matplotlib.pyplot as plt

n_samples = 100
rng = np.random.RandomState(20090425)
ms = np.array([[-1, -1.5], [1, 1]])
ps = np.array([0.2, 0.8])

zs = np.array([rng.multinomial(1, ps) for _ in range(n_samples)]).T
xs = [z[:, np.newaxis] * rng.multivariate_normal(m, np.eye(2), size=n_samples)
      for z, m in zip(zs, ms)]
data = np.sum(np.dstack(xs), axis=2)

plt.figure(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], c='g', alpha=0.5)
plt.scatter(ms[0, 0], ms[0, 1], c='r', s=100)
plt.scatter(ms[1, 0], ms[1, 1], c='b', s=100)

Gaussian mixture models are usually constructed with **categorical random variables**. However, ADVI cannot fit models with discrete variables, since it uses the gradient of the model with respect to the parameters. 

To get around this, we can integrate (analytically) over the latent group indicators. This results in a continuous probability distribution that is the weighted sum of the Gaussian components. The log likelihood of the total probability is calculated using `logsumexp` (LSE), which is a [standard technique for making this kind of calculation stable](https://arxiv.org/abs/1506.03431):

$$\text{logSumExp}(x) = \log \left[ \sum_{i=1}^N x_i - \max(x) \right] + \max(x)$$

In the below code, DensityDist class is used as the likelihood term. The second argument, `logp_gmix(mus, pi, np.eye(2))`, is a Python function which recieves observations (denoted by `value`) and returns the tensor representation of the log-likelihood. 

In [ ]:
from pymc3.math import logsumexp

# Log likelihood of normal distribution
def logp_normal(mu, tau, value):
    # log probability of individual samples
    k = tau.shape[0]
    delta = lambda mu: value - mu
    return (-1 / 2.) * (k * tt.log(2 * np.pi) + tt.log(1./det(tau)) +
                         (delta(mu).dot(tau) * delta(mu)).sum(axis=1))

# Log likelihood of Gaussian mixture distribution
def logp_gmix(mus, pi, tau):
    def logp_(value):        
        logps = [tt.log(pi[i]) + logp_normal(mu, tau, value)
                 for i, mu in enumerate(mus)]
            
        return tt.sum(logsumexp(tt.stacklists(logps)[:, :n_samples], axis=0))

    return logp_

with pm.Model() as model:
    μ = [MvNormal('μ_%d' % i, mu=np.zeros(2), tau=0.1 * np.eye(2), shape=(2,))
           for i in range(2)]
    π = Dirichlet('π', a=0.1 * np.ones(2), shape=(2,))
    x = DensityDist('x', logp_gmix(μ, π, np.eye(2)), observed=data)

For comparison with ADVI, run MCMC. 

In [ ]:
with model:
    trace = sample(10000, step=Metropolis(), init=None)

Check posterior of component means and weights. We can see that the MCMC samples of the component means differed in variance due to the difference of the sample size of these clusters. 

In [ ]:
plt.figure(figsize=(5, 5))
mu_0, mu_1 = trace['μ_0', 500:], trace['μ_1', 500:]
plt.scatter(mu_0[:, 0], mu_0[:, 1], alpha=0.1)
plt.scatter(mu_1[:, 0], mu_1[:, 1], alpha=0.1)
plt.scatter(data[:, 0], data[:, 1], c='k', alpha=0.3)

In [ ]:
sns.barplot([1, 2], np.mean(trace['π'][-5000:], axis=0))

We can fit the same model with ADVI as follows. 

In [ ]:
with model:

    means, sds, elbos = pm.advi(n=10000, learning_rate=1e-1)

The function returns three variables. `means` and `sds` are the mean and standard deviations of the variational posterior (*Note that these values are in the transformed space, not in the original space*). 

But, we can see the variational posterior in the original space. 

In [ ]:
from copy import deepcopy

mu_0, sd_0 = means['μ_0'], sds['μ_0']
mu_1, sd_1 = means['μ_1'], sds['μ_1']

def logp_normal_np(mu, tau, value):
    # log probability of individual samples
    k = tau.shape[0]
    delta = lambda mu: value - mu
    return (-1 / 2.) * (k * np.log(2 * np.pi) + np.log(1./np.linalg.det(tau)) +
                         (delta(mu).dot(tau) * delta(mu)).sum(axis=1))

def threshold(zz):    
    zz_ = deepcopy(zz)
    zz_[zz < np.max(zz) * 1e-2] = None
    return zz_

def plot_logp_normal(ax, mu, sd, cmap):
    f = lambda value: np.exp(logp_normal_np(mu, np.diag(1 / sd**2), value))
    g = lambda mu, sd: np.arange(mu - 3, mu + 3, .1)
    xx, yy = np.meshgrid(g(mu[0], sd[0]), g(mu[1], sd[1]))
    zz = f(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).reshape(xx.shape)
    ax.contourf(xx, yy, threshold(zz), cmap=cmap, alpha=0.9)
           
fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], alpha=0.3, c='k')
plot_logp_normal(ax, mu_0, sd_0, cmap='winter')
plot_logp_normal(ax, mu_1, sd_1, cmap='summer')

`elbos` contains the trace of the evidence lower bound, showing stochastic convergence of the algorithm. 

In [ ]:
plt.plot(elbos)

To demonstrate that ADVI works for large dataset with mini-batch, let's create 100,000 samples from the same mixture distribution. 

In [ ]:
n_samples = 100000

zs = np.array([rng.multinomial(1, ps) for _ in range(n_samples)]).T
xs = [z[:, np.newaxis] * rng.multivariate_normal(m, np.eye(2), size=n_samples)
      for z, m in zip(zs, ms)]
data = np.sum(np.dstack(xs), axis=2)

plt.figure(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], c='k', alpha=0.01)
plt.scatter(ms[0, 0], ms[0, 1], s=100)
plt.scatter(ms[1, 0], ms[1, 1], s=100)

In [ ]:
with pm.Model() as model:
    
    μ = [MvNormal('μ_%d' % i, mu=np.zeros(2), tau=0.1 * np.eye(2), shape=(2,))
           for i in range(2)]
    π = Dirichlet('π', a=0.1 * np.ones(2), shape=(2,))
    x = DensityDist('x', logp_gmix(μ, π, np.eye(2)), observed=data)
    
    trace = sample(10000, step=Metropolis(), init=None)

Posterior samples are concentrated on the true means, so looks like single point for each component. 

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], alpha=0.1, c='k')
mu_0, mu_1 = trace['μ_0'], trace['μ_1']
plt.scatter(mu_0[-500:, 0], mu_0[-500:, 1], alpha=0.4)
plt.scatter(mu_1[-500:, 0], mu_1[-500:, 1], alpha=0.4)

For ADVI with mini-batch, pass a Theano `tensor` to the likelihood (an `ObservedRV`). The tensor will iteratively be replaced with mini-batches during the ADVI run. Because of the difference of the size of mini-batch and whole samples, the log-likelihood term needs to be appropriately scaled. 

In [ ]:
data_t = tt.matrix()
data_t.tag.test_value = np.zeros((1, 2)).astype(float)

with pm.Model() as model:
    mu = [MvNormal('mu_%d' % i, mu=np.zeros(2), tau=0.1 * np.eye(2), shape=(2,))
           for i in range(2)]
    pi = Dirichlet('pi', a=0.1 * np.ones(2), shape=(2,))
    x = DensityDist('x', logp_gmix(mu, pi, np.eye(2)), observed=data_t)
    
minibatch_tensors = [data_t]
minibatch_RVs = [xs]

Make a generator for mini-batches, each of size 200. Mini-batches are generated using Theano `RandomState` realizations. 

In [ ]:
def create_minibatch(data):
    rng = np.random.RandomState(0)
    
    while True:
        ixs = rng.randint(len(data), size=200)
        yield [data[ixs]]

minibatches = create_minibatch(data)
total_size = len(data)

The ADVI model fitting is much faster than MCMC, 

In [ ]:
with model:
    pm.advi_minibatch(n=1000, minibatch_tensors=minibatch_tensors, 
    minibatch_RVs=minibatch_RVs, minibatches=minibatches,
    total_size=total_size, learning_rate=1e-1)
    

... but the result is almost the same (at least for this simple model). 

In [ ]:
from copy import deepcopy

mu_0, sd_0 = means['mu_0'], sds['mu_0']
mu_1, sd_1 = means['mu_1'], sds['mu_1']

fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(data[:, 0], data[:, 1], alpha=0.2, c='k')
plt.scatter(mu_0[0], mu_0[1])
plt.scatter(mu_1[0], mu_1[1])

The variance of the trace of ELBO is larger than without mini-batch because of the subsampling from the whole samples. 

In [ ]:
plt.plot(elbos)

## References

1.	[Kucukelbir A, Ranganath R, Gelman A, Blei DM.](https://arxiv.org/abs/1506.03431) Automatic Variational Inference in Stan. arXiv. 2015;stat.ML.